In [15]:
### COMBINER ###

from pandas import DataFrame
from tkinter import filedialog , Tk
from openpyxl import load_workbook
from re import search


##################################################################################################
# PARTE 1 - Selezione di tutti i file con cui lavorare

window =  Tk().withdraw()

initial_dir = r"X:\listino prezzi\TENDER 2021"

# Creare lista (prima tupla) con nome dei file da combinare
t_file_names = filedialog.askopenfilenames(title = 'Choose files to combine',
                                           initialdir = initial_dir,
                                           filetypes = (('XLSX', '*.xlsx'),
                                                        ('XLSM', '*.xlsm'),
                                                        ('All files', '*.*')))
# .xls non supportato da openpyxl
# La selezione non serve a niente

file_names = list(t_file_names) # Convertire tupla in lista



##################################################################################################
# PARTE 2 - Riconoscere file internal

# Estrarre nome del file internal dalla lista di file da combinare
pos = [ind for ind, name in enumerate(file_names) if search('._Internal.xls.', name)]
        
base = file_names.pop(pos[0])
base_wb = load_workbook(base, keep_vba=True)
# NB: Usando pop(), la lista file_names non contiene più il nome del file internal



##################################################################################################
# PARTE 3 - Combinare i file con i prezzi tramite Pandas e salvare in xlsx/xlsm


# Funzione per rimuovere spazi bianchi da formattazione Ticontract
def remove_spaces(worksheet):
    iterator = worksheet.iter_rows(max_row = max_row,
                                   max_col = max_col)
    for row in iterator:
        for cell in row:
            if cell.value == ' ':
                cell.value = None


for sheetname in base_wb.sheetnames:  # Loop per nome worksheet
    
    base_ws = base_wb[sheetname]

    max_row = base_ws.max_row
    max_col = base_ws.max_column
    
    remove_spaces(base_ws)
   
    # Inserire in una lista l'elenco dei range delle celle unite dal file base
    # ci servirà alla fine per ricalcare la formattazione del file internal        
    merged_cells = [str(item) for item in base_ws.merged_cells]
    
    # Unmerge tutte le celle unite
    #(necessario altrimenti non fa sovrascrivere valori alla fine)    
    for rng in merged_cells:
        base_ws.unmerge_cells(rng)
        
    # Creare base_df
    base_df = DataFrame(base_ws.values)                 
            

    for file in file_names:  # Loop per nome file

        # Caricare il ws specifico
        wb = load_workbook(file, keep_vba=True)  # flag 'keep_vba=True' --> serve?
        ws = wb[sheetname]
        
        remove_spaces(ws)  
           
        # Creazione df in pandas e combinazione con base_df
        df = DataFrame(ws.values)  
        base_df = base_df.combine_first(df)
        
        
    # Necessario ridefinire l'iterator per il prossimo punto (non so perché)    
    iterator = base_ws.iter_rows(max_row = max_row,
                                 max_col = max_col)

    # Metti valori del df combinato nell'apposito base_ws
    for row in iterator:
        for cell in row:
             cell.value = base_df.iloc[cell.row-1][cell.column-1]  #Pandas conta da 0, non 1
            
    # Re-merge celle sul file finale
    for rng in merged_cells:
        base_ws.merge_cells(rng)
        
    
# Salvare il file output con nuovo nome

output_name = base.replace('Internal', 'Internal_c')

base_wb.save(output_name)